<a href="https://colab.research.google.com/github/Thiagokiam/CNN_Models_DeepLearning/blob/main/C%C3%B3pia_de_Model_TMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 as cv
import time, os
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

import torch
from torch import nn, optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
class ImagemTensor(Dataset):
  def __init__(self, dados, scaler_feat=None, scaler_label=None):

    self.dados = dados

  def __getitem__(self, idx):

    sample = self.dados[0][idx]
    label  = self.dados[1][idx]

    return sample, label

  def __len__(self):
    return len(self.dados[0])

In [ ]:
# Configurando hiperparâmetros.
args = {
    'epoch_num': 1000,        # Número de épocas.
    'lr': 1e-9,             # Taxa de aprendizado.
    'weight_decay': 1e-9,   # Penalidade L2 (Regularização).
    'batch_size': 10,        # Tamanho do batch.
    'num_workers': 5,       # Quantos treinamentos simultâneos.
}
if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

In [ ]:
# Carregamento dos dados de atributos e rótulos
atributos = torch.load('Tensores/atributos_random_tensor_n_aleatorio.pt')
rotulos = torch.load('Tensores/rotulos_tensor_n_aleatorio.pt')


In [ ]:
# dataset_treino = ImagemTensor([atributos[:], rotulos[:]])
# dataset_teste  = ImagemTensor([atributos[:], rotulos[:]])
dataset_treino = ImagemTensor([atributos[:4200], rotulos[:4200]])
dataset_teste  = ImagemTensor([atributos[4200:6000], rotulos[4200:6000]])

In [ ]:

dataloader_treino = DataLoader(dataset_treino,
                               batch_size = args['batch_size'],
                               shuffle=False,
                               num_workers = args['num_workers'])

dataloader_teste = DataLoader(dataset_teste,
                              batch_size = args['batch_size'],
                              shuffle=False,
                              num_workers = args['num_workers'])

In [ ]:
class CNN_Regressor(nn.Module):

  def __init__(self):   # Lembra que no init é obrigatório inicializar a super classe. Agora podemos fazer o que quisermos, que no caso do init é a definição da nossa arquitetura.
    super(cnnclassifier, self).__init__()

    #          ConvBlock 1
    self.cnn       = nn.Conv2d(1, 256, kernel_size=3, stride=1, padding=0)
    self.batch     = nn.BatchNorm2d(256)
    self.relu      = nn.ReLU()
    self.avgpool   = nn.AvgPool2d(kernel_size=4, stride=4, padding=0)

    #          ConvBlock 2
    self.cnn2      = nn.Conv2d(256, 192, kernel_size=3, stride=1, padding=0)
    self.batch2    = nn.BatchNorm2d(192)
    self.relu2     = nn.ReLU()
    self.maxpool2  = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
    #          ConvBlock 3
    self.cnn3      = nn.Conv2d(192, 128, kernel_size=3, stride=1, padding=1)
    self.batch3    = nn.BatchNorm2d(128)
    self.relu3     = nn.ReLU()
    self.maxpool3  = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)

    #           Fully Connected
    self.flatten    = nn.Flatten()
    self.linear     = nn.Linear(100352,320)
    self.relu4      = nn.ReLU()
    self.linear2    = nn.Linear(320,1)

  def forward(self, X):
  #          CNN 1
    out = self.cnn(X)
    out = self.batch(out)
    out = self.relu(out)
    out = self.avgpool(out)

  #          CNN 2

    out = self.cnn2(out)
    out = self.batch2(out)
    out = self.relu2(out)
    out = self.maxpool2(out)

  #         CNN 3

    out = self.cnn3(out)
    out = self.batch3(out)
    out = self.relu3(out)
    out = self.maxpool3(out)

  #         FC1

    out = self.flatten(out)
    out = self.linear(out)
    out = self.relu4(out)
    out = self.linear2(out)

    return out


net = CNN_Regressor()
net = net.to(args['device'])


In [ ]:
if torch.cuda.device_count() >= 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  modelparallel = nn.DataParallel(net)

modelparallel.to(args['device'])
print(modelparallel)

In [ ]:
#criterion = nn.L1Loss().to(args['device'])
criterion = nn.HuberLoss().to(args['device'])
#criterion = nn.MSELoss().to(args['device'])

#optimizer = optim.SGD(modelparallel.parameters(), lr=args['lr'], momentum=0.9, weight_decay=args['weight_decay'])
optimizer = optim.Adam(modelparallel.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

In [ ]:
def train(dataloader_treino, modelparallel, epoch):

  # Training mode
  modelparallel.train()

  start = time.time()

  epoch_loss  = []
  pred_list, rotulo_list = [], []
  for batch in dataloader_treino:

    dados, rotulo = batch

    # Cast data in GPU
    dados = dados.to(args['device'])
    rotulo = rotulo.to(args['device'])

    # Forward
    ypred = modelparallel(dados)
    loss = criterion(ypred, rotulo)
    epoch_loss.append(loss.cpu().data)

    _, pred = torch.max(ypred, axis=1)
    pred_list.append(pred.cpu().numpy())
    rotulo_list.append(rotulo.cpu().numpy())

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  epoch_loss = np.asarray(epoch_loss)
  pred_list  = np.asarray(pred_list).ravel()
  rotulo_list  = np.asarray(rotulo_list).ravel()



  end = time.time()
  print('#################### Train ####################')
  print('Epoch %d, Loss: %.4f +/- %.4f, Time: %.2f' % (epoch, epoch_loss.mean(), epoch_loss.std(), end-start))


  return epoch_loss.mean()

In [ ]:
def validate(dataloader_teste, modelparallel, epoch):

  # Evaluation mode
  modelparallel.eval()

  start = time.time()

  epoch_loss  = []
  pred_list, rotulo_list = [], []
  with torch.no_grad():
    for batch in dataloader_teste:

      dados, rotulo = batch

      # Cast do dado na GPU
      dados = dados.to(args['device'])
      rotulo = rotulo.to(args['device'])

      # Forward
      ypred = net(dados)
      loss = criterion(ypred, rotulo)
      epoch_loss.append(loss.cpu().data)

      _, pred = torch.max(ypred, axis=1)
      pred_list.append(pred.cpu().numpy())
      rotulo_list.append(rotulo.cpu().numpy())

  epoch_loss = np.asarray(epoch_loss)

  end = time.time()
  print('********** Validate **********')
  print('Epoch %d, Loss: %.4f +/- %.4f, Time: %.2f\n' % (epoch, epoch_loss.mean(), epoch_loss.std(), end-start))
  return epoch_loss.mean()

train_losses, test_losses = [], []
for epoch in range(args['epoch_num']):

  # Train
  train_losses.append(train(dataloader_treino, net, epoch))

  # Validate
  test_losses.append(validate(dataloader_teste, net, epoch))

In [ ]:
df = pd.DataFrame({'Treino': train_losses, 'Teste': test_losses})

In [ ]:
df.to_excel('ModelCNN/VGG_adam_ReLU30%dadosteste_SGD_e-9_pesoe-9_STACK.xlsx')

In [ ]:
#print(train_losses)
plt.plot(test_losses, 'b', label = "Teste")
plt.plot(train_losses, 'orange', label = "Treino")
plt.ylabel('Loss')
plt.xlabel("Épocas")

In [ ]:
#print(train_losses)
plt.plot(test_losses, 'b', label = "Teste")
plt.plot(train_losses, 'orange', label = "Treino")
plt.ylabel('Loss')
plt.xlabel("Épocas")

In [ ]:
def previsao_alexnet(ypred, rotulo):
  y = (10**np.asarray(ypred).ravel())/1000
  r = (10**np.asarray(rotulo).ravel())/1000
  k_p = np.log10(y)
  k_g = np.log10(r)

  N = len(k_p)

  soma = np.sum((k_p-k_g)**2)
  raiz = np.sqrt(soma/N)

  sigma = 10**(raiz)

  return y, r, sigma

In [ ]:
y_pred, rotulo_pred, erro_sigma = previsao_alexnet(predicao, rotulo_lis )

In [ ]:
plt.subplots(figsize = (6,4))

sns.scatterplot(x = rotulo_pred,
                y = y_pred)

reta = pd.DataFrame({'x' : np.arange(100),
                     'y' : np.arange(100)})
sns.lineplot(data = reta,
               x = 'x',
               y = 'y')

plt.yscale('log')
plt.xscale('log')
plt.xlabel('Permeabilidade Gás (mD)')
plt.ylabel('Predição (mD)')
plt.title('Permeabilidada gás x Predição')
plt.show()


In [ ]:
df_predicoes = pd.DataFrame({'Predição': y_pred, 'Rotulo': rotulo_pred, 'Erro sigma':erro_sigma})

In [ ]:
#Save data
df_predicoes.to_excel('ModelCNN/predição_TMM_ADAM_HUBER_tentativa2maxpool_ReLU30%dadosteste_e-9_STACK.xlsx')

In [ ]:
#Save model
torch.save(modelparallel, 'ModelCNN/model_TMM_pytorch_ADAM_30%dadosteste_HUBER_e-9_STACK.pt')

In [ ]:
#Load model
modelparallel_load = torch.load('ModelCNN/model_TMM_pytorch_ADAM_30%dadosteste_HUBER_e-9_STACK.pt')

modelparallel_load.eval()